# Chapter 2-1

# 完全データ①

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

table_answer = pd.DataFrame({
    "a": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    "b": [1, 2, 3, 4, 5, 6, 7, 0, 0,  0, 11, 12, 13, 14, 15, 16, 17, 18,  0,  0,  0,  0,  0,  0],
    "c": [1, 2, 3, 4, 5, 0, 0, 8, 9,  0, 11, 12, 13, 14,  0,  0,  0,  0, 19, 20, 21, 22,  0,  0],
    "d": [1, 2, 3, 0, 0, 6, 7, 8, 9,  0, 11, 12,  0,  0, 15, 16,  0,  0, 19, 20,  0,  0, 23, 24],
})

display(table_answer)
sns.heatmap(table_answer, vmin=0, vmax=1, cmap="jet", cbar=False)

In [ ]:
# Tracking table
table_a_b = pd.DataFrame({
    'a': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'b': [1, 2, 3, 4, 5, 6, 7, 0, 0,  0],
})

table_a_c = pd.DataFrame({
    'a': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'c': [1, 2, 3, 4, 5, 0, 0, 6, 7,  0],
}).fillna(0)

table_a_d = pd.DataFrame({
    'a': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'd': [1, 2, 3, 0, 0, 4, 5, 6, 7,  0],
}).fillna(0)

table_b_c = pd.DataFrame({
    'b': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'c': [1, 2, 3, 4, 5, 0, 0, 8, 9, 10, 11,  0,  0,  0,  0],
}).fillna(0)

table_b_d = pd.DataFrame({
    'b': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'd': [1, 2, 3, 0, 0, 4, 5, 8, 9,  0,  0, 10, 11,  0,  0],
}).fillna(0)

table_c_d = pd.DataFrame({
    'c': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'd': [1, 2, 3, 0, 0, 6, 7, 8, 9,  0,  0, 12, 13,  0,  0],
}).fillna(0)

display(table_a_b)
display(table_a_c)
display(table_a_d)
display(table_b_c)
display(table_b_d)
display(table_c_d)

list_table = [table_a_b, table_a_c, table_a_d, table_b_c, table_b_d, table_c_d]

dict_answer = {
    "a": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    "b": [1, 2, 3, 4, 5, 6, 7, 0, 0,  0,  8,  9, 10, 11, 12, 13, 14, 15,  0,  0,  0,  0,  0,  0],
    "c": [1, 2, 3, 4, 5, 0, 0, 6, 7,  0,  8,  9, 10, 11,  0,  0,  0,  0, 12, 13, 14, 15,  0,  0],
    "d": [1, 2, 3, 0, 0, 4, 5, 6, 7,  0,  8,  9,  0,  0, 10, 11,  0,  0, 12, 13,  0,  0, 14, 15],
}

display(pd.DataFrame(dict_answer).fillna(0))

In [ ]:
dict_array = {
    "a": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]),
    "b": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]),
    "c": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]),
    "d": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]),
}

plt.figure()
for x in dict_array.keys():
    for y in dict_array[x]:
        plt.scatter(x, y, color="black")
for table in list_table:
    x_1, x_2 = table.columns
    for idx, row in table.iterrows():
        y_1, y_2 = row[x_1], row[x_2]
        if y_1 == 0 or y_2 == 0:
            continue
        if ord(x_2) - ord(x_1) == 1:
            color="red"
            delta = -0.1
        elif ord(x_2) - ord(x_1) == 2:
            color="blue"
            delta = 0
        elif ord(x_2) - ord(x_1) == 3:
            color="green"
            delta = 0.1
        plt.plot([x_1, x_2], [y_1+delta, y_2+delta], color=color)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

list_table = [table_a_b, table_a_c, table_a_d, table_b_c, table_b_d, table_c_d]

G = nx.Graph()

for session in dict_array.keys():
    for roi in dict_array[session]:
        G.add_node(f"{session}_{roi}")


for table in list_table:
    session_pri, session_sec = table.columns
    for _, row in table.dropna().iterrows():
        roi_pri, roi_sec = row[session_pri], row[session_sec]
        if roi_pri == 0 or roi_sec == 0:
            continue
        node1 = f"{session_pri}_{roi_pri}"
        node2 = f"{session_sec}_{roi_sec}"
        G.add_edge(node1, node2)

plt.figure(figsize=(10, 10))
pos = nx.spring_layout(G, k=0.3, iterations=50)
nx.draw(G, pos=pos, with_labels=True, node_size=100, font_size=10, font_color="blue", node_color="red", edge_color="black")
plt.show()

In [ ]:
# 部分完全グラフ(クリーク)の抽出
list_clique = list(nx.find_cliques(G))

display(list_clique)
table_clique = pd.DataFrame(columns=sorted(list(set(pd.concat(list_table, axis=1).columns))))

for idx, clique in enumerate(list_clique):
    for node in clique:
        session, roi = node.split("_")
        table_clique.loc[idx, session] = roi
table_clique = table_clique.sort_values(by=table_clique.columns.tolist())
table_clique.index = np.arange(len(table_clique))
display(table_clique)

table_tracking = table_clique.copy()

for idx, row in table_tracking.iterrows():
    for session in row.index:
        if pd.isnull(row[session]):
            table_tracking.loc[idx, session] = 0
        else:
            table_tracking.loc[idx, session] = idx + 1

table_tracking = table_tracking.astype("int16")
display(table_tracking)

In [ ]:
sns.heatmap(table_answer, cmap="jet", cbar=False, vmin=0, vmax=1)
plt.show()

sns.heatmap(table_tracking, cmap="jet", cbar=False, vmin=0, vmax=1)
plt.show()

# 完全データ②

In [ ]:
# Tracking table
table_a_b = pd.DataFrame({
    'a': [1, 2, 3, 4, 5,  6, 7, 8, 9, 10],
    'b': [0, 4, 0, 1, 10, 0, 5, 8, 0, 11],
})

table_a_c = pd.DataFrame({
    'a': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'c': [0, 5, 2, 1, 0, 0, 0, 8, 0, 11],
}).fillna(0)

table_a_d = pd.DataFrame({
    'a': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'd': [5, 3, 4, 2, 6, 0, 0, 8, 0,  0],
}).fillna(0)

table_b_c = pd.DataFrame({
    'b': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'c': [1, 0, 0, 5, 0, 9, 0, 8, 6,  0, 11],
}).fillna(0)

table_b_d = pd.DataFrame({
    'b': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'd': [2, 0, 0, 3, 0, 0, 1, 8, 7,  6,  0],
}).fillna(0)

table_c_d = pd.DataFrame({
    'c': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'd': [2, 4, 0, 0, 3, 7, 0, 8, 0,  9,  0,  0],
}).fillna(0)

display(table_a_b)
display(table_a_c)
display(table_a_d)
display(table_b_c)
display(table_b_d)
display(table_c_d)

list_table = [table_a_b, table_a_c, table_a_d, table_b_c, table_b_d, table_c_d]

dict_answer = {
    "a": [1, 2, 3, 4, 5,  6, 7, 8, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0],
    "b": [0, 4, 0, 1, 10, 0, 5, 8, 0, 11, 2, 3, 6, 7, 9, 0, 0, 0, 0,  0,  0],
    "c": [0, 5, 2, 1, 0,  0, 0, 8, 0, 11, 0, 0, 9, 0, 6, 3, 4, 7, 9, 10, 12],
    "d": [5, 3, 4, 2, 6,  0, 0, 8, 0,  0, 0, 0, 0, 1, 7, 0, 0, 0, 0,  9,  0],
}
table_answer = pd.DataFrame(dict_answer).fillna(0)
display(table_answer)

In [ ]:
dict_array = {
    "a": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]),
    "b": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]),
    "c": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    "d": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
}

plt.figure()
for x in dict_array.keys():
    for y in dict_array[x]:
        plt.scatter(x, y, color="black")
for table in list_table:
    x_1, x_2 = table.columns
    for idx, row in table.iterrows():
        y_1, y_2 = row[x_1], row[x_2]
        if y_1 == 0 or y_2 == 0:
            continue
        if ord(x_2) - ord(x_1) == 1:
            color="red"
            delta = -0.1
        elif ord(x_2) - ord(x_1) == 2:
            color="blue"
            delta = 0
        elif ord(x_2) - ord(x_1) == 3:
            color="green"
            delta = 0.1
        plt.plot([x_1, x_2], [y_1+delta, y_2+delta], color=color)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

list_table = [table_a_b, table_a_c, table_a_d, table_b_c, table_b_d, table_c_d]

G = nx.Graph()

for session in dict_array.keys():
    for roi in dict_array[session]:
        G.add_node(f"{session}_{roi}")


for table in list_table:
    session_pri, session_sec = table.columns
    for _, row in table.dropna().iterrows():
        roi_pri, roi_sec = row[session_pri], row[session_sec]
        if roi_pri == 0 or roi_sec == 0:
            continue
        node1 = f"{session_pri}_{roi_pri}"
        node2 = f"{session_sec}_{roi_sec}"
        G.add_edge(node1, node2)

plt.figure(figsize=(10, 10))
pos = nx.spring_layout(G, k=0.3, iterations=50)
nx.draw(G, pos=pos, with_labels=True, node_size=100, font_size=10, font_color="blue", node_color="red", edge_color="black")
plt.show()

In [ ]:
# 部分完全グラフ(クリーク)の抽出
list_clique = list(nx.find_cliques(G))
display(list_clique)

table_clique = pd.DataFrame(columns=sorted(list(set(pd.concat(list_table, axis=1).columns))))

for idx, clique in enumerate(list_clique):
    for node in clique:
        session, roi = node.split("_")
        table_clique.loc[idx, session] = roi
table_clique = table_clique.fillna(0).astype("int16")
# sort by column, and replace 0 to inf
table_clique = table_clique.sort_values(
    by=table_clique.columns.tolist(), 
    key=lambda col: col.apply(lambda x: float('inf') if x == 0 else x) 
    )
table_clique.index = np.arange(len(table_clique))

display(table_clique)

In [ ]:
sns.heatmap(table_answer, cmap="jet", cbar=False)
plt.show()

sns.heatmap(table_clique, cmap="jet", cbar=False)
plt.show()

# 不完全データ

In [ ]:
# Tracking table
table_a_b = pd.DataFrame({
    'a': [1, 2, 3, 4, 5,  6, 7, 8, 9, 10],
    'b': [0, 4, 0, 1, 0,  0, 5, 8, 0, 11],
})

table_a_c = pd.DataFrame({
    'a': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'c': [0, 5, 2, 0, 0, 0, 0, 8, 0, 11],
}).fillna(0)

table_a_d = pd.DataFrame({
    'a': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'd': [5, 3, 4, 2, 6, 0, 0, 8, 0,  0],
}).fillna(0)

table_b_c = pd.DataFrame({
    'b': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'c': [1, 0, 0, 5, 0, 6, 0, 8, 6,  0, 11],
}).fillna(0)

table_b_d = pd.DataFrame({
    'b': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'd': [2, 0, 0, 3, 0, 0, 1, 8, 7,  6,  0],
}).fillna(0)

table_c_d = pd.DataFrame({
    'c': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'd': [2, 4, 0, 0, 3, 7, 0, 8, 0,  9,  0,  0],
}).fillna(0)

display(table_a_b)
display(table_a_c)
display(table_a_d)
display(table_b_c)
display(table_b_d)
display(table_c_d)

list_table = [table_a_b, table_a_c, table_a_d, table_b_c, table_b_d, table_c_d]

dict_answer = {
    "a": [1, 2, 3, 4, 5,  6, 7, 8, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0],
    "b": [0, 4, 0, 1, 10, 0, 5, 8, 0, 11, 2, 3, 6, 7, 9, 0, 0, 0, 0,  0,  0],
    "c": [0, 5, 2, 1, 0,  0, 0, 8, 0, 11, 0, 0, 9, 0, 6, 3, 4, 7, 9, 10, 12],
    "d": [5, 3, 4, 2, 6,  0, 0, 8, 0,  0, 0, 0, 0, 1, 7, 0, 0, 0, 0,  9,  0],
}
table_answer = pd.DataFrame(dict_answer).fillna(0)
display(table_answer)

In [ ]:
dict_array = {
    "a": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]),
    "b": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]),
    "c": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    "d": np.array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
}

plt.figure()
for x in dict_array.keys():
    for y in dict_array[x]:
        plt.scatter(x, y, color="black")
for table in list_table:
    x_1, x_2 = table.columns
    for idx, row in table.iterrows():
        y_1, y_2 = row[x_1], row[x_2]
        if y_1 == 0 or y_2 == 0:
            continue
        if ord(x_2) - ord(x_1) == 1:
            color="red"
            delta = -0.1
        elif ord(x_2) - ord(x_1) == 2:
            color="blue"
            delta = 0
        elif ord(x_2) - ord(x_1) == 3:
            color="green"
            delta = 0.1
        plt.plot([x_1, x_2], [y_1+delta, y_2+delta], color=color)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

list_table = [table_a_b, table_a_c, table_a_d, table_b_c, table_b_d, table_c_d]

G = nx.Graph()

for session in dict_array.keys():
    for roi in dict_array[session]:
        G.add_node(f"{session}_{roi}")


for table in list_table:
    session_pri, session_sec = table.columns
    for _, row in table.dropna().iterrows():
        roi_pri, roi_sec = row[session_pri], row[session_sec]
        if roi_pri == 0 or roi_sec == 0:
            continue
        node1 = f"{session_pri}_{roi_pri}"
        node2 = f"{session_sec}_{roi_sec}"
        G.add_edge(node1, node2)

plt.figure(figsize=(10, 10))
pos = nx.spring_layout(G, k=0.3, iterations=50)
nx.draw(G, pos=pos, with_labels=True, node_size=100, font_size=10, font_color="blue", node_color="red", edge_color="black")
plt.show()

In [ ]:
# 部分完全グラフ(クリーク)の抽出
list_clique = []
list_incomplete_subgraph = []

# get connected components
for nodes_subgraph in nx.connected_components(G):
    subG = G.subgraph(nodes_subgraph).copy()
    # judge whether the subgraph is a complete graph
    n_nodes_subG = len(subG.nodes)
    n_edges_subG = len(subG.edges)
    if n_nodes_subG == 1:
        list_clique.append(list(subG.nodes))
    else:
        if n_edges_subG == n_nodes_subG * (n_nodes_subG - 1) / 2:
            list_clique.append(list(subG.nodes))
        else:
            list_incomplete_subgraph.append(list(subG.nodes))
print("Complete subgraphs")
display(list_clique)
print("Incomplete subgraphs")
display(list_incomplete_subgraph)

table_clique = pd.DataFrame(columns=sorted(list(set(pd.concat(list_table, axis=1).columns))))

for idx, clique in enumerate(list_clique):
    for node in clique:
        session, roi = node.split("_")
        table_clique.loc[idx, session] = roi
table_clique = table_clique.fillna(0).astype("int16")
# sort by column, and replace 0 to inf
table_clique = table_clique.sort_values(
    by=table_clique.columns.tolist(), 
    key=lambda col: col.apply(lambda x: float('inf') if x == 0 else x) 
    )
table_clique.index = np.arange(len(table_clique))

display(table_clique)

In [ ]:
sns.heatmap(table_answer, cmap="jet", cbar=False)
plt.show()

sns.heatmap(table_clique, cmap="jet", cbar=False)
plt.show()